In [1]:
import os

In [2]:
%pwd

'c:\\Users\\paisa\\OneDrive\\Desktop\\MLOPS\\kidney-disease-classification\\notebooks'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\paisa\\OneDrive\\Desktop\\MLOPS\\kidney-disease-classification'

In [ ]:
os.environ["MLFLOW_URI"] = "<add your mlflow uri here>"
os.environ["MLFLOW_TRACKING_USERNAME"] = "<add your username here>"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "<add your password here>"

In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.keras")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from src.CNN_classifier.constants import *
from src.CNN_classifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.keras",
            training_data="artifacts/data_ingestion/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone",
            mlflow_uri="https://dagshub.com/paisatvik/kidney-disease-classification.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [16]:
import mlflow.tensorflow


class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model,artifact_path="model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model,artifact_path="model")


In [17]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

2025-06-18 03:14:27,887 - INFO - common - yaml file: config\config.yaml loaded successfully
2025-06-18 03:14:27,889 - INFO - common - yaml file: params.yaml loaded successfully
2025-06-18 03:14:27,889 - INFO - common - created directory at: artifacts
Found 134 images belonging to 2 classes.


c:\Users\paisa\OneDrive\Desktop\MLOPS\kidney-disease-classification\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 9s 997ms/step - accuracy: 0.7657 - loss: 7.0900
2025-06-18 03:14:37,484 - INFO - common - json file saved at: scores.json


2025/06/18 03:14:38 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/06/18 03:14:48 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\paisa\AppData\Local\Temp\tmpgsb94w16\model, flavor: keras). Fall back to return ['keras==3.10.0']. Set logging level to DEBUG to see the full traceback. 
2025/06/18 03:14:48 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/06/18 03:15:02 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.


🏃 View run illustrious-yak-997 at: https://dagshub.com/paisatvik/kidney-disease-classification.mlflow/#/experiments/0/runs/3de5363f53ce45c496d94c8e76969109
🧪 View experiment at: https://dagshub.com/paisatvik/kidney-disease-classification.mlflow/#/experiments/0
